## I. Data Preparation

In this notebook, we shall prepare data to be used for inference.

We shall start by importing the libraries and data. Then I shall explore the structure of the data set and proceed to:
* Feature engineering (turning non numerical variables into categorical and creating dummies)
* Outlier handling

Finally, I save the modified dataset for creating and testing a model for inference purposes. 

### 1.Importing libraries

In [1]:
# import libraries
import pandas as pd
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn import linear_model
import sklearn.metrics as sklm
import numpy as np
import numpy.random as nr
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as ss
import math
from glob import glob


%matplotlib inline
%matplotlib inline

### 2. Import datasets 

In [2]:
df=pd.read_csv('Credit-Scoring-Clean.csv')
df.shape

(950, 21)

In [3]:
# REmove duplicate columns if any
#Remove duplicate columns
_, i = np.unique(df.columns, return_index=True)
df=df.iloc[:, i]
df.shape

(950, 21)

We can see there were no duplicate rows

### 3. Data Exploration

In [4]:
df.head(5)

,Age,CheckingAcctStat,CreditAmount,CreditHistory,CreditStatus,Duration,Employment,ExistingCreditsAtBank,ForeignWorker,Housing,...,Job,NumberDependents,OtherDetorsGuarantors,OtherInstalments,PresentResidenceTime,Property,Purpose,Savings,SexAndStatus,Telephone
0,0.089286,A11,0.161770,A32,1,0.205882,A73,0.0,A201,A152,...,A173,0.0,A101,A143,0.333333,A121,A43,A61,A92,A191
1,0.125000,A14,0.058380,A32,0,0.294118,A74,0.0,A201,A152,...,A173,0.0,A101,A143,0.666667,A122,A43,A62,A94,A192
2,0.232143,A14,0.069055,A32,0,0.205882,A73,0.0,A201,A153,...,A174,0.0,A101,A143,0.333333,A124,A43,A61,A93,A192
3,0.214286,A11,0.358094,A32,1,0.647059,A73,0.0,A201,A152,...,A173,0.0,A101,A143,0.333333,A123,A43,A61,A92,A192
4,0.178571,A13,0.023825,A33,0,0.029412,A72,0.0,A201,A152,...,A173,0.0,A101,A141,0.000000,A122,A43,A61,A92,A191


In [5]:
# see if there is any missing value
df.isna().sum()

Age                      0
CheckingAcctStat         0
CreditAmount             0
CreditHistory            0
CreditStatus             0
Duration                 0
Employment               0
ExistingCreditsAtBank    0
ForeignWorker            0
Housing                  0
InstallmentRatePecnt     0
Job                      0
NumberDependents         0
OtherDetorsGuarantors    0
OtherInstalments         0
PresentResidenceTime     0
Property                 0
Purpose                  0
Savings                  0
SexAndStatus             0
Telephone                0
dtype: int64

Data seems to be (as expected by the name of the dataset), clean and ready for inference

### 3.1 Label exploration

Let's look at the label, and turn it into YES/NO, and then into 0/1 values

In [6]:
df.CreditStatus.value_counts()

1    525
0    425
Name: CreditStatus, dtype: int64

Data is slightly imbalanced, so I shall undertake action to balance it

In [7]:
df.CreditStatus.describe()

count    950.000000
mean       0.552632
std        0.497484
min        0.000000
25%        0.000000
50%        1.000000
75%        1.000000
max        1.000000
Name: CreditStatus, dtype: float64

### 3. Feature selection and engineering

#### 3.1 Feature selection

In [8]:
#show the different columns in the dataset
df.columns

Index(['Age', 'CheckingAcctStat', 'CreditAmount', 'CreditHistory',
       'CreditStatus', 'Duration', 'Employment', 'ExistingCreditsAtBank',
       'ForeignWorker', 'Housing', 'InstallmentRatePecnt', 'Job',
       'NumberDependents', 'OtherDetorsGuarantors', 'OtherInstalments',
       'PresentResidenceTime', 'Property', 'Purpose', 'Savings',
       'SexAndStatus', 'Telephone'],
      dtype='object')

In [9]:
#Drop columns that won't be used on this model 
df = df.drop(['Housing','SexAndStatus', 'OtherDetorsGuarantors',
              'ExistingCreditsAtBank','OtherInstalments', 'CreditAmount',
              'Employment','ForeignWorker','Job'], axis=1)

In [10]:
#Shape of remaining dataset
df.shape

(950, 12)

#### 3.2 Feature Engineering

In [11]:
# view column types
df.dtypes

Age                     float64
CheckingAcctStat         object
CreditHistory            object
CreditStatus              int64
Duration                float64
InstallmentRatePecnt    float64
NumberDependents        float64
PresentResidenceTime    float64
Property                 object
Purpose                  object
Savings                  object
Telephone                object
dtype: object

I need to turn the object type variables into categorical. Eventually, if there aren't too many values in an single category, I could aggregate them.  

In [12]:
# See values per categorical value
df['CheckingAcctStat'].value_counts()

A11    333
A14    301
A12    266
A13     50
Name: CheckingAcctStat, dtype: int64

In [13]:
df['CreditHistory'].value_counts()

A32    525
A34    234
A33     76
A31     62
A30     53
Name: CreditHistory, dtype: int64

In [14]:
df['Property'].value_counts()

A123    329
A121    239
A122    220
A124    162
Name: Property, dtype: int64

In [15]:
df['Purpose'].value_counts()

A43     254
A40     254
A42     153
A49      96
A41      84
A46      44
A45      32
A44      11
A48      11
A410     11
Name: Purpose, dtype: int64

In [16]:
df['Savings'].value_counts()

A61    618
A65    146
A62     95
A63     54
A64     37
Name: Savings, dtype: int64

In [17]:
df['Telephone'].value_counts()

A191    573
A192    377
Name: Telephone, dtype: int64

The followoing cells will create dummie variables for the non numerical columns we have just explored, and then drop the column used to create the dummy variables (this way we prevent collinearity)

In [18]:
#create dummies for categorical columns
df = pd.concat([df,pd.get_dummies(df['CheckingAcctStat'])],axis=1)
df = pd.concat([df,pd.get_dummies(df['CreditHistory'])],axis=1)
df = pd.concat([df,pd.get_dummies(df['Property'])],axis=1)
df = pd.concat([df,pd.get_dummies(df['Purpose'])],axis=1)
df = pd.concat([df,pd.get_dummies(df['Savings'])],axis=1)
df = pd.concat([df,pd.get_dummies(df['Telephone'])],axis=1)


In [19]:
df=df.drop(['CheckingAcctStat',
       'CreditHistory', 'Property','Purpose',
       'Savings',
       'Telephone'], axis=1)

In [20]:
#See data shape before handling outliers
df.shape

(950, 36)

#### 3.3 Handling outliers

In [21]:
df.dtypes

Age                     float64
CreditStatus              int64
Duration                float64
InstallmentRatePecnt    float64
NumberDependents        float64
PresentResidenceTime    float64
A11                       uint8
A12                       uint8
A13                       uint8
A14                       uint8
A30                       uint8
A31                       uint8
A32                       uint8
A33                       uint8
A34                       uint8
A121                      uint8
A122                      uint8
A123                      uint8
A124                      uint8
A40                       uint8
A41                       uint8
A410                      uint8
A42                       uint8
A43                       uint8
A44                       uint8
A45                       uint8
A46                       uint8
A48                       uint8
A49                       uint8
A61                       uint8
A62                       uint8
A63     

In [22]:
quant_features = ['Age','Duration','InstallmentRatePecnt','NumberDependents','PresentResidenceTime']

In [23]:
# Handle outliers on numeric features
import scipy as sp
from scipy import stats
df = df[(np.abs(sp.stats.zscore(df[quant_features])) < 3).all(axis=1)]

In [24]:
# See data shape after handling outliers
df.shape

(937, 36)

950 - 937 = 13 is the number of rows, thereofre the number of outliers deleted from the dataset when handling outliers

Finally, I save the modified data set to create and test a model in a separate notebook

In [25]:
#export data set to main directory to be used in a different jupyter notebook
df.to_csv("dfprepared.csv", index=False)